In [1]:
from GenZ import decode_moddeling, prefill_moddeling

from GenZ.system import System
import pandas as pd
from GenZ.analyse_model import get_model_df, get_summary_table

from GenZ import create_inference_mamba_prefix_model, create_inference_mamba_decode_model

In [2]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2   

In [3]:
system = System(frequency=1000 , flops=2000, off_chip_mem_size=(80*1024), compute_efficiency=0.8, memory_efficiency=0.8,
                    offchip_mem_bw=3500, bits='int8', external_mem_bw=128, interchip_link_bw=256, interchip_link_latency=2)

In [9]:
################################################################################################## #
### Model Characterization Calculation
################################################################################################## #
model_prefill = create_inference_mamba_prefix_model(input_sequence_length=1000,output_gen_tokens = 0 ,
                                    name='tiiuae/falcon-mamba-7b')

B = 1
model_df = get_model_df(model_prefill, system, batch_size=B,  model_characterstics = True)
summary_table = get_summary_table(model_df, model_characterstics = True)


LogL: 3


In [6]:
from GenZ.Models.utils import parse_einsum_expression

In [7]:
L = 1000
S = 16
F = 4096
logL= 3
parse_einsum_expression('lbfs,lbs->lbf', (logL,'b',F,S), (L,'b',S), (logL,'b',F))

ValueError: Dimension mismatch for subscript: l, Got: 1000, Expected: 3

In [ ]:
display(model_df)
display(summary_table)

In [ ]:
import matplotlib.pyplot as plt

# Initialize a dictionary to store Op Intensity for each Op Type
op_intensity_data = {op_idx: [] for op_idx in range(len(model_df))}

# Vary B from 1 to 128
batch_sizes = [1,2,3,4,8,16,32,64,128]
for B in batch_sizes:
    model_df = get_model_df(model_prefill, system, batch_size=B, model_characterstics=True)
    for op_idx in range(len(model_df)):
        subset = model_df.loc[op_idx]
        op_intensity_data[op_idx].append(subset['Op Intensity'])

# Plotting
plt.figure(figsize=(10, 6))
for op_type, intensities in op_intensity_data.items():
    plt.plot(batch_sizes, intensities, marker='o', linestyle='-', label=op_type)

plt.xlabel('Batch Size (B)')
plt.ylabel('Op Intensity')
plt.title('Op Intensity of Each Op Type as a Function of B')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
################################################################################################## #
### Model Characterization Calculation
################################################################################################## #
model_decode = create_inference_mamba_decode_model(input_sequence_length=10000,output_gen_tokens = 0 ,
                                    name='tiiuae/falcon-mamba-7b')

B = 1
model_df = get_model_df(model_decode, system, batch_size=B,  model_characterstics = True)
summary_table = get_summary_table(model_df, model_characterstics = True)


In [ ]:
display(model_df)
display(summary_table)

In [ ]:
import matplotlib.pyplot as plt

# Initialize a dictionary to store Op Intensity for each Op Type
op_intensity_data = {op_idx: [] for op_idx in range(len(model_df))}

# Vary B from 1 to 128
batch_sizes = [1,2,3,4,8,16,32,64,128]
for B in batch_sizes:
    model_df = get_model_df(model_decode, system, batch_size=B, model_characterstics=True)
    for op_idx in range(len(model_df)):
        subset = model_df.loc[op_idx]
        op_intensity_data[op_idx].append(subset['Op Intensity'])

# Plotting
plt.figure(figsize=(10, 6))
for op_type, intensities in op_intensity_data.items():
    plt.plot(batch_sizes, intensities, marker='o', linestyle='-', label=op_type)

plt.xlabel('Batch Size (B)')
plt.ylabel('Op Intensity')
plt.title('Op Intensity of Each Op Type as a Function of B')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model_df